In [1]:
from io import StringIO
import tempfile
import warnings

import numpy as np
import pandas as pd

import MDAnalysis as mda
import MDAnalysis.analysis.rdf, MDAnalysis.analysis.msd

import openmm
import openmm.app
import openmm.unit

In [2]:
offxml = """
<ForceField>
<AtomTypes>
 <Type name="0" class="Na" element="Na" mass="22.990"/>
 <Type name="1" class="Cl" element="Cl" mass="35.453"/>
 ...
</AtomTypes>

<Residues>
 <Residue name="Na">
  <Atom name="Na" type="0"/>
 </Residue>
 <Residue name="Cl">
  <Atom name="Cl" type="1"/>
 </Residue>
</Residues>

<NonbondedForce coulomb14scale="0.0" lj14scale="0.0">
 <Atom type="0" charge="1.0" sigma="0.3330" epsilon="0.011598"/>
 <Atom type="1" charge="-1.0" sigma="0.4417" epsilon="0.493712"/>
</NonbondedForce>
</ForceField>
"""

forcefield = openmm.app.forcefield.ForceField(StringIO(offxml))

In [3]:
def perform_nacl_md(temperature, pressure, nsteps,
                    timestep = 0.001 * openmm.unit.picoseconds,
                    friction = 1. / openmm.unit.picoseconds,
                    report_frequency = 50):
    """
    Perform a MD simulation of NaCl 4x4x4 supercell in the NPT ensemble,
    at specific temperature and pressure. Return the data and trajectory.
    """

    # Read the starting configuration
    pdb = openmm.app.pdbfile.PDBFile('NaCl_4x4x4.pdb')

    # Check that the force field covers all atom types in the input structure
    unmatched_residues = forcefield.getUnmatchedResidues(pdb.topology)
    assert len(unmatched_residues) == 0

    # Create the system and add a barostat
    system = forcefield.createSystem(pdb.topology, nonbondedMethod=openmm.app.CutoffPeriodic, nonbondedCutoff=1.1)
    barostat = openmm.MonteCarloBarostat(pressure, temperature)
    system.addForce(barostat)

    # Specify how the integration should be performed
    integrator = openmm.LangevinMiddleIntegrator(temperature, friction, timestep)
    simulation = openmm.app.Simulation(pdb.topology, system, integrator)

    # Set up the initial state: positions and velocities
    simulation.context.setPositions(pdb.positions)
    simulation.minimizeEnergy()
    simulation.context.setVelocitiesToTemperature(temperature)

    # Write trajectory
    dcd_reporter = openmm.app.DCDReporter(f'md_trajectory.dcd', report_frequency, enforcePeriodicBox=False)
    simulation.reporters.append(dcd_reporter)

    # Output numerical data
    data_reporter = openmm.app.StateDataReporter(f'md_data.csv', report_frequency,
        step=True, time=True, potentialEnergy=True, kineticEnergy=True,
        volume=True, temperature=True, density=True,
    )
    simulation.reporters.append(data_reporter)

    # Run the simulation
    simulation.step(nsteps)

    # Terminate the simulation, close output files (needed on Windows)
    del simulation
    del dcd_reporter
    del data_reporter

    # Gather the numerical data
    data = pd.read_csv(f'md_data.csv')

    # Read the trajectory
    with warnings.catch_warnings():
        # Ignore "DeprecationWarning: DCDReader currently makes independent timesteps by copying self.ts while other readers update self.ts inplace"
        warnings.filterwarnings("ignore", category=DeprecationWarning)
        traj = mda.Universe('NaCl_4x4x4.pdb', f'md_trajectory.dcd')
        traj.transfer_to_memory()

    # Return data and trajectory
    return data, traj

In [4]:
data, traj = perform_nacl_md(300, 0, 1000)

In [5]:
len(traj.trajectory)

20

In [6]:
data.describe()

,"#""Step""",Time (ps),Potential Energy (kJ/mole),Kinetic Energy (kJ/mole),Temperature (K),Box Volume (nm^3),Density (g/mL)
count,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000
mean,525.000000,0.525000,-153163.632313,1239.022690,194.416408,11.881607,2.091735
std,295.803989,0.295804,159.645584,172.808607,27.115588,0.232355,0.041563
min,50.000000,0.050000,-153390.386249,845.643250,132.690809,11.421027,2.050696
25%,287.500000,0.287500,-153270.842922,1132.836029,177.754543,11.702068,2.059509
50%,525.000000,0.525000,-153204.118523,1269.921875,199.264833,11.982246,2.073404
75%,762.500000,0.762500,-153071.614972,1342.962616,210.725736,12.063072,2.123065
max,1000.000000,1.000000,-152796.637037,1505.651123,236.253368,12.114914,2.175286
